# Statistical Analysis & Parameter Tuning for VRPTW Solver

This notebook performs comprehensive statistical analysis of the VRPTW solver by:
1. Testing different parameter configurations
2. Analyzing the impact of each parameter on time-windowed problems
3. Visualizing results with plots
4. Exporting data to CSV for further analysis

## Parameters to analyze:
- Initial temperature (SA)
- Cooling rate (alpha)
- Tabu tenure
- Iterations per temperature

## Note:
VRPTW is more constrained than CVRP, so parameter sensitivity may differ.

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import vrplib
import yaml
import random
import math
import time
import os
from collections import deque
from copy import deepcopy
from typing import List, Dict, Tuple, Optional
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Load base configuration
with open('config.yaml', 'r') as f:
    base_config = yaml.safe_load(f)

print("Base configuration loaded!")

Base configuration loaded!


## Copy Solver Functions

We need the solver components from the VRPTW notebook.

In [3]:
# VRPTWSolution class
class VRPTWSolution:
    def __init__(self, routes: List[List[int]], distance_matrix: np.ndarray,
                 time_windows: np.ndarray, service_times: np.ndarray):
        self.routes = routes
        self.distance_matrix = distance_matrix
        self.time_windows = time_windows
        self.service_times = service_times
        self.route_times = []
        self.cost = self.calculate_cost()
    
    def calculate_route_times(self, route: List[int]) -> List[float]:
        if not route:
            return []
        times = []
        current_time = 0.0
        current_location = 0
        for customer in route:
            travel_time = self.distance_matrix[current_location, customer]
            arrival_time = current_time + travel_time
            ready_time = self.time_windows[customer, 0]
            start_service = max(arrival_time, ready_time)
            times.append(arrival_time)
            current_time = start_service + self.service_times[customer]
            current_location = customer
        return times
    
    def calculate_cost(self) -> float:
        total_cost = 0
        self.route_times = []
        for route in self.routes:
            if len(route) == 0:
                self.route_times.append([])
                continue
            times = self.calculate_route_times(route)
            self.route_times.append(times)
            total_cost += self.distance_matrix[0, route[0]]
            for i in range(len(route) - 1):
                total_cost += self.distance_matrix[route[i], route[i+1]]
            total_cost += self.distance_matrix[route[-1], 0]
        return total_cost
    
    def update_cost(self):
        self.cost = self.calculate_cost()
    
    def is_time_feasible(self, route: List[int]) -> bool:
        if not route:
            return True
        current_time = 0.0
        current_location = 0
        for customer in route:
            travel_time = self.distance_matrix[current_location, customer]
            arrival_time = current_time + travel_time
            due_date = self.time_windows[customer, 1]
            if arrival_time > due_date:
                return False
            ready_time = self.time_windows[customer, 0]
            start_service = max(arrival_time, ready_time)
            current_time = start_service + self.service_times[customer]
            current_location = customer
        return_time = current_time + self.distance_matrix[current_location, 0]
        depot_due = self.time_windows[0, 1]
        return return_time <= depot_due
    
    def is_feasible(self, demands: np.ndarray, capacity: int) -> bool:
        for route in self.routes:
            route_demand = sum(demands[customer] for customer in route)
            if route_demand > capacity:
                return False
            if not self.is_time_feasible(route):
                return False
        return True
    
    def copy(self):
        return VRPTWSolution([route.copy() for route in self.routes],
                            self.distance_matrix, self.time_windows, self.service_times)

print("VRPTWSolution class defined!")

VRPTWSolution class defined!


In [4]:
# Helper functions
def calculate_distance_matrix(instance: Dict) -> np.ndarray:
    if 'edge_weight' in instance:
        return instance['edge_weight']
    elif 'node_coord' in instance:
        coords = np.array(instance['node_coord'])
        n = len(coords)
        dist_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    dist = np.sqrt(np.sum((coords[i] - coords[j])**2))
                    dist_matrix[i, j] = dist
        return dist_matrix
    else:
        raise ValueError("Instance must have either 'edge_weight' or 'node_coord'")

def nearest_neighbor_solution_tw(instance: Dict, distance_matrix: np.ndarray,
                                  time_windows: np.ndarray, service_times: np.ndarray, config: Dict) -> VRPTWSolution:
    n_customers = instance['dimension'] - 1
    capacity = instance['capacity']
    demands = np.array(instance['demand'])
    unvisited = set(range(1, n_customers + 1))
    routes = []
    
    while unvisited:
        route = []
        current_load = 0
        current_time = 0.0
        current_location = 0
        
        while True:
            best_customer = None
            best_score = float('inf')
            
            for customer in unvisited:
                if current_load + demands[customer] > capacity:
                    continue
                travel_time = distance_matrix[current_location, customer]
                arrival_time = current_time + travel_time
                due_date = time_windows[customer, 1]
                if arrival_time > due_date:
                    continue
                ready_time = time_windows[customer, 0]
                start_service = max(arrival_time, ready_time)
                finish_time = start_service + service_times[customer]
                return_time = finish_time + distance_matrix[customer, 0]
                depot_due = time_windows[0, 1]
                if return_time > depot_due:
                    continue
                dist = distance_matrix[current_location, customer]
                urgency = due_date - arrival_time
                score = dist - 0.1 * urgency
                randomness = config['initial_solution']['randomness']
                score *= (1 + randomness * random.random())
                if score < best_score:
                    best_score = score
                    best_customer = customer
            
            if best_customer is None:
                break
            route.append(best_customer)
            current_load += demands[best_customer]
            travel_time = distance_matrix[current_location, best_customer]
            arrival_time = current_time + travel_time
            ready_time = time_windows[best_customer, 0]
            start_service = max(arrival_time, ready_time)
            current_time = start_service + service_times[best_customer]
            current_location = best_customer
            unvisited.remove(best_customer)
        
        if route:
            routes.append(route)
    
    return VRPTWSolution(routes, distance_matrix, time_windows, service_times)

print("Helper functions defined!")

Helper functions defined!


In [5]:
# Simplified neighborhood operators for speed
def relocate_operator_tw(solution: VRPTWSolution, demands: np.ndarray, capacity: int) -> Optional[VRPTWSolution]:
    best_solution = None
    best_cost = solution.cost
    for i in range(len(solution.routes)):
        route_i = solution.routes[i]
        if len(route_i) == 0:
            continue
        for pos_i in range(len(route_i)):
            for j in range(len(solution.routes)):
                if i == j:
                    continue
                for pos_j in range(len(solution.routes[j]) + 1):
                    new_solution = solution.copy()
                    removed = new_solution.routes[i].pop(pos_i)
                    new_solution.routes[j].insert(pos_j, removed)
                    if new_solution.is_feasible(demands, capacity):
                        new_solution.update_cost()
                        if new_solution.cost < best_cost:
                            best_cost = new_solution.cost
                            best_solution = new_solution
    return best_solution

def two_opt_operator_tw(solution: VRPTWSolution, demands: np.ndarray, capacity: int) -> Optional[VRPTWSolution]:
    best_solution = None
    best_cost = solution.cost
    for route_idx in range(len(solution.routes)):
        route = solution.routes[route_idx]
        if len(route) < 2:
            continue
        for i in range(len(route) - 1):
            for j in range(i + 1, len(route)):
                new_solution = solution.copy()
                new_solution.routes[route_idx][i:j+1] = list(reversed(new_solution.routes[route_idx][i:j+1]))
                if new_solution.is_time_feasible(new_solution.routes[route_idx]):
                    new_solution.update_cost()
                    if new_solution.cost < best_cost:
                        best_cost = new_solution.cost
                        best_solution = new_solution
    return best_solution

def vnd_tw(solution: VRPTWSolution, demands: np.ndarray, capacity: int, config: Dict) -> VRPTWSolution:
    neighborhoods = {'relocate': relocate_operator_tw, 'two_opt': two_opt_operator_tw}
    neighborhood_order = ['relocate', 'two_opt']
    max_no_improve = 30  # Reduced for speed
    current_solution = solution
    k = 0
    no_improve_count = 0
    while k < len(neighborhood_order) and no_improve_count < max_no_improve:
        neighborhood_name = neighborhood_order[k]
        operator = neighborhoods[neighborhood_name]
        new_solution = operator(current_solution, demands, capacity)
        if new_solution is not None and new_solution.cost < current_solution.cost:
            current_solution = new_solution
            k = 0
            no_improve_count = 0
        else:
            k += 1
            no_improve_count += 1
    return current_solution

class TabuList:
    def __init__(self, tenure: int):
        self.tenure = tenure
        self.tabu_dict = {}
        self.current_iteration = 0
    def add(self, move: Tuple, tenure_variation: int = 0):
        actual_tenure = self.tenure + random.randint(-tenure_variation, tenure_variation)
        self.tabu_dict[move] = self.current_iteration + actual_tenure
    def is_tabu(self, move: Tuple) -> bool:
        if move not in self.tabu_dict:
            return False
        return self.tabu_dict[move] > self.current_iteration
    def increment_iteration(self):
        self.current_iteration += 1
        expired = [move for move, expiration in self.tabu_dict.items() 
                   if expiration <= self.current_iteration]
        for move in expired:
            del self.tabu_dict[move]

def acceptance_probability(current_cost: float, new_cost: float, temperature: float) -> float:
    if new_cost < current_cost:
        return 1.0
    return math.exp((current_cost - new_cost) / temperature)

print("Simplified operators defined!")

Simplified operators defined!


In [6]:
# Simplified solver for parameter testing
def solve_with_config_tw(instance_path: str, config: Dict, time_limit: float = 60) -> Dict:
    random.seed(config['general']['random_seed'])
    np.random.seed(config['general']['random_seed'])
    
    instance = vrplib.read_instance(instance_path, instance_format='solomon')
    distance_matrix = calculate_distance_matrix(instance)
    demands = np.array(instance['demand'])
    capacity = instance['capacity']
    time_windows = np.array(instance['time_window'])
    service_times = np.array(instance['service_time'])
    
    initial_solution = nearest_neighbor_solution_tw(instance, distance_matrix, time_windows, service_times, config)
    improved_solution = vnd_tw(initial_solution, demands, capacity, config)
    
    # Simplified SA
    temp = config['simulated_annealing']['initial_temperature']
    final_temp = config['simulated_annealing']['final_temperature']
    alpha = config['simulated_annealing']['alpha']
    iterations_per_temp = config['simulated_annealing']['iterations_per_temperature']
    tabu_tenure = config['tabu_search']['tabu_tenure']
    tabu_list = TabuList(tabu_tenure)
    
    current_solution = improved_solution
    best_solution = current_solution.copy()
    operators = [relocate_operator_tw, two_opt_operator_tw]
    
    start_time = time.time()
    iterations = 0
    
    while temp > final_temp and (time.time() - start_time) < time_limit:
        for _ in range(iterations_per_temp):
            operator = random.choice(operators)
            new_solution = operator(current_solution, demands, capacity)
            if new_solution is None:
                continue
            if random.random() < acceptance_probability(current_solution.cost, new_solution.cost, temp):
                current_solution = new_solution
                if current_solution.cost < best_solution.cost:
                    best_solution = current_solution.copy()
            iterations += 1
        temp *= alpha
    
    elapsed_time = time.time() - start_time
    
    optimal_cost = None
    sol_path = instance_path.replace('.txt', '.sol')
    if os.path.exists(sol_path):
        try:
            optimal_solution = vrplib.read_solution(sol_path)
            optimal_cost = optimal_solution['cost']
        except:
            pass
    
    gap = None
    if optimal_cost:
        gap = ((best_solution.cost - optimal_cost) / optimal_cost) * 100
    
    return {'cost': best_solution.cost, 'optimal_cost': optimal_cost, 'gap': gap, 'time': elapsed_time, 'iterations': iterations}

print("Simplified VRPTW solver defined!")

Simplified VRPTW solver defined!


## Select Test Instances

In [7]:
# Select test instances (small Solomon instances)
solomon_dir = 'data/cvrplib/Vrp-Set-Solomon'
if os.path.exists(solomon_dir):
    all_instances = [os.path.join(solomon_dir, f) for f in os.listdir(solomon_dir) if f.endswith('.txt')]
else:
    all_instances = [os.path.join('data', f) for f in os.listdir('data') if f.endswith('.txt') and not f.startswith('empty')]

# Select C101 and R101 (representative instances)
test_instances = [inst for inst in all_instances if any(name in inst for name in ['C101.txt', 'R101.txt'])]

print(f"Selected {len(test_instances)} test instances:")
for inst in test_instances:
    print(f"  - {os.path.basename(inst)}")

Selected 3 test instances:
  - C101.txt
  - R101.txt
  - RC101.txt


## Experiment 1: Impact of Initial Temperature

In [8]:
print("Experiment 1: Testing different initial temperatures...\n")

temperature_values = [500, 1000, 2000]
results_temp = []

for instance_path in test_instances:
    instance_name = os.path.basename(instance_path)
    print(f"Testing {instance_name}...")
    
    for temp in temperature_values:
        test_config = deepcopy(base_config)
        test_config['simulated_annealing']['initial_temperature'] = temp
        result = solve_with_config_tw(instance_path, test_config, time_limit=30)
        results_temp.append({
            'instance': instance_name,
            'initial_temperature': temp,
            'cost': result['cost'],
            'optimal_cost': result['optimal_cost'],
            'gap': result['gap'],
            'time': result['time']
        })
        print(f"  Temp={temp}: Cost={result['cost']:.2f}, Gap={result['gap']:.2f}% (if known)")
    print()

df_temp = pd.DataFrame(results_temp)
print("\nExperiment 1 completed!")

Experiment 1: Testing different initial temperatures...

Testing C101.txt...


KeyError: 'dimension'

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for instance in df_temp['instance'].unique():
    data = df_temp[df_temp['instance'] == instance]
    axes[0].plot(data['initial_temperature'], data['cost'], marker='o', label=instance)
axes[0].set_xlabel('Initial Temperature')
axes[0].set_ylabel('Solution Cost')
axes[0].set_title('Cost vs Initial Temperature (VRPTW)')
axes[0].legend()
axes[0].grid(True)

df_temp_with_gap = df_temp[df_temp['gap'].notna()]
if not df_temp_with_gap.empty:
    for instance in df_temp_with_gap['instance'].unique():
        data = df_temp_with_gap[df_temp_with_gap['instance'] == instance]
        axes[1].plot(data['initial_temperature'], data['gap'], marker='o', label=instance)
    axes[1].axhline(y=7, color='r', linestyle='--', label='Target 7%')
    axes[1].set_xlabel('Initial Temperature')
    axes[1].set_ylabel('Gap (%)')
    axes[1].set_title('Gap vs Initial Temperature (VRPTW)')
    axes[1].legend()
    axes[1].grid(True)

plt.tight_layout()
plt.savefig('analysis_results/plots/vrptw_experiment1_temperature.png', dpi=300, bbox_inches='tight')
plt.show()
df_temp.to_csv('analysis_results/csv/vrptw_experiment1_temperature.csv', index=False)
print("Results saved!")

## Experiment 2: Impact of Cooling Rate (Alpha)

In [ ]:
print("Experiment 2: Testing different cooling rates...\n")

alpha_values = [0.90, 0.95, 0.97]
results_alpha = []

for instance_path in test_instances:
    instance_name = os.path.basename(instance_path)
    print(f"Testing {instance_name}...")
    
    for alpha in alpha_values:
        test_config = deepcopy(base_config)
        test_config['simulated_annealing']['alpha'] = alpha
        result = solve_with_config_tw(instance_path, test_config, time_limit=30)
        results_alpha.append({
            'instance': instance_name,
            'alpha': alpha,
            'cost': result['cost'],
            'optimal_cost': result['optimal_cost'],
            'gap': result['gap'],
            'time': result['time']
        })
        print(f"  Alpha={alpha}: Cost={result['cost']:.2f}, Gap={result['gap']:.2f}% (if known)")
    print()

df_alpha = pd.DataFrame(results_alpha)
print("\nExperiment 2 completed!")

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for instance in df_alpha['instance'].unique():
    data = df_alpha[df_alpha['instance'] == instance]
    axes[0].plot(data['alpha'], data['cost'], marker='o', label=instance)
axes[0].set_xlabel('Cooling Rate (Alpha)')
axes[0].set_ylabel('Solution Cost')
axes[0].set_title('Cost vs Cooling Rate (VRPTW)')
axes[0].legend()
axes[0].grid(True)

df_alpha_with_gap = df_alpha[df_alpha['gap'].notna()]
if not df_alpha_with_gap.empty:
    for instance in df_alpha_with_gap['instance'].unique():
        data = df_alpha_with_gap[df_alpha_with_gap['instance'] == instance]
        axes[1].plot(data['alpha'], data['gap'], marker='o', label=instance)
    axes[1].axhline(y=7, color='r', linestyle='--', label='Target 7%')
    axes[1].set_xlabel('Cooling Rate (Alpha)')
    axes[1].set_ylabel('Gap (%)')
    axes[1].set_title('Gap vs Cooling Rate (VRPTW)')
    axes[1].legend()
    axes[1].grid(True)

plt.tight_layout()
plt.savefig('analysis_results/plots/vrptw_experiment2_alpha.png', dpi=300, bbox_inches='tight')
plt.show()
df_alpha.to_csv('analysis_results/csv/vrptw_experiment2_alpha.csv', index=False)
print("Results saved!")

## Experiment 3: Impact of Tabu Tenure

In [ ]:
print("Experiment 3: Testing different tabu tenures...\n")

tenure_values = [10, 20, 30]
results_tenure = []

for instance_path in test_instances:
    instance_name = os.path.basename(instance_path)
    print(f"Testing {instance_name}...")
    
    for tenure in tenure_values:
        test_config = deepcopy(base_config)
        test_config['tabu_search']['tabu_tenure'] = tenure
        result = solve_with_config_tw(instance_path, test_config, time_limit=30)
        results_tenure.append({
            'instance': instance_name,
            'tabu_tenure': tenure,
            'cost': result['cost'],
            'optimal_cost': result['optimal_cost'],
            'gap': result['gap'],
            'time': result['time']
        })
        print(f"  Tenure={tenure}: Cost={result['cost']:.2f}, Gap={result['gap']:.2f}% (if known)")
    print()

df_tenure = pd.DataFrame(results_tenure)
print("\nExperiment 3 completed!")

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for instance in df_tenure['instance'].unique():
    data = df_tenure[df_tenure['instance'] == instance]
    axes[0].plot(data['tabu_tenure'], data['cost'], marker='o', label=instance)
axes[0].set_xlabel('Tabu Tenure')
axes[0].set_ylabel('Solution Cost')
axes[0].set_title('Cost vs Tabu Tenure (VRPTW)')
axes[0].legend()
axes[0].grid(True)

df_tenure_with_gap = df_tenure[df_tenure['gap'].notna()]
if not df_tenure_with_gap.empty:
    for instance in df_tenure_with_gap['instance'].unique():
        data = df_tenure_with_gap[df_tenure_with_gap['instance'] == instance]
        axes[1].plot(data['tabu_tenure'], data['gap'], marker='o', label=instance)
    axes[1].axhline(y=7, color='r', linestyle='--', label='Target 7%')
    axes[1].set_xlabel('Tabu Tenure')
    axes[1].set_ylabel('Gap (%)')
    axes[1].set_title('Gap vs Tabu Tenure (VRPTW)')
    axes[1].legend()
    axes[1].grid(True)

plt.tight_layout()
plt.savefig('analysis_results/plots/vrptw_experiment3_tenure.png', dpi=300, bbox_inches='tight')
plt.show()
df_tenure.to_csv('analysis_results/csv/vrptw_experiment3_tenure.csv', index=False)
print("Results saved!")

## Summary Report

In [ ]:
print("\n" + "="*70)
print("VRPTW STATISTICAL ANALYSIS SUMMARY")
print("="*70)

all_experiments = [
    ('Temperature', df_temp),
    ('Alpha', df_alpha),
    ('Tabu Tenure', df_tenure)
]

for exp_name, df in all_experiments:
    print(f"\n{exp_name} Experiment:")
    print("-" * 40)
    df_with_gap = df[df['gap'].notna()]
    if not df_with_gap.empty:
        print(f"  Average gap: {df_with_gap['gap'].mean():.2f}%")
        print(f"  Best gap: {df_with_gap['gap'].min():.2f}%")
        print(f"  Worst gap: {df_with_gap['gap'].max():.2f}%")
        print(f"  Std deviation: {df_with_gap['gap'].std():.2f}%")
    else:
        print("  No gap data available")
    print(f"  Average time: {df['time'].mean():.2f}s")

print("\n" + "="*70)
print("\nAnalysis complete! Results saved to analysis_results/")

In [ ]:
# Find and save best parameters
recommendations = {}

if not df_temp[df_temp['gap'].notna()].empty:
    best_temp = df_temp[df_temp['gap'].notna()].groupby('initial_temperature')['gap'].mean().idxmin()
    recommendations['initial_temperature'] = float(best_temp)
    print(f"Recommended Initial Temperature: {best_temp}")

if not df_alpha[df_alpha['gap'].notna()].empty:
    best_alpha = df_alpha[df_alpha['gap'].notna()].groupby('alpha')['gap'].mean().idxmin()
    recommendations['alpha'] = float(best_alpha)
    print(f"Recommended Alpha: {best_alpha}")

if not df_tenure[df_tenure['gap'].notna()].empty:
    best_tenure = df_tenure[df_tenure['gap'].notna()].groupby('tabu_tenure')['gap'].mean().idxmin()
    recommendations['tabu_tenure'] = int(best_tenure)
    print(f"Recommended Tabu Tenure: {best_tenure}")

with open('analysis_results/recommended_config_vrptw.yaml', 'w') as f:
    yaml.dump(recommendations, f)
    
print("\nRecommendations saved to: analysis_results/recommended_config_vrptw.yaml")